<a id='imp'></a>
# 2. Importing Libraries

In [1]:
!pip install arch
import math
import datetime
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from ipywidgets import HBox, VBox
from tabulate import tabulate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance, plot_tree
#Matplotlib style
plt.style.use('fivethirtyeight')
#Ignoring some warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.5/927.5 kB 26.5 MB/s eta 0:00:00


<a id='rea'></a>
# 3. Reading the data and preprocesing

Data Source: https://www.kaggle.com/datasets/lucastrenzado/repsol-stock-data-20-years
<br/><br/>
We will only use the "close" column of our dataset

Calculating the return of each day:

% Change in a day = $(\frac{P_2}{P_1}-1) \times 100\%$


In [ ]:
#Read the data as a pandas DataFrame
repsol_stock_raw_data = pd.read_csv("/kaggle/input/repsol-stock-data-20-years/RepsolStockData20Years.csv")
#Calculate the percentage change by day
returns_repsol = 100 * repsol_stock_raw_data.close.pct_change().dropna()
#Drop 0 results, there is a error in the dataset and for 74 days,the stock market was close, so the return is 0
returns_repsol = returns_repsol.drop(returns_repsol[returns_repsol == 0].index)
#Display the 3 first rows of the Serie
returns_repsol.head(3)

<a id='eda'></a>
# 4. EDA: Visualizing Returns & Volatility

We have obtained the daily returns for Repsol stock from October 29, 2002 to October 25, 2022.

In [ ]:
#Average of return column
returns_repsol.mean()

The mean of the daily % returns is 0.0223%, but we will not use that information, as we will be studying the volatility.

In [ ]:
#Using the raw close prices to plot the evolution of the stock
close_prices = pd.DataFrame(repsol_stock_raw_data["close"])
#By dividing each close price by the first price in our dataset we calculate the accumulated return for each day
cum_rets = close_prices / close_prices.iloc[0,:]
#Using the plotly.express module we can plot our newly created cum_rets
fig = px.line(cum_rets.iloc[:,:], width=1000, height=500)
#Adding Title
fig.update_layout(title_text='Cumulative Return of Repsol Stock (Not Including Dividends) from 2002-10-28 to 2022-10-28')
#This will print the graph
fig.show()

## 4.1 Measuring Volatility

In [ ]:
#The daily volatility is the std of the returns
daily_volatility = returns_repsol.std()
#The monthly volatility is the result of multiplying the daily vol * square root of 21, this is because there are 21 trading days in a month
monthly_volatility = math.sqrt(21) * daily_volatility
#The annual volatility is the result of multiplying the daily vol * square root of 252, this is because there are 252 trading days in a year
annual_volatility = math.sqrt(252) * daily_volatility
#Using tabulate package we can print a nice table
print(tabulate([['Repsol',daily_volatility,monthly_volatility,annual_volatility]],headers = ['Daily Volatility %', 'Monthly Volatility %', 'Annual Volatility %'],tablefmt = 'fancy_grid',stralign='center',numalign='center',floatfmt=".2f"))

Plotting daily returns

In [ ]:
#We can plot the daily retuns of repsol using a line graph using .plot from pandas
returns_repsol.plot(figsize =(16,5), title = 'Repsol daily returns');

In [ ]:
#We create a distribution plot using plotly.figure_factory, we reshape the data to have them in a vector
return_dist_plot  = ff.create_distplot([returns_repsol.values.reshape(-1)], group_labels = [' '])
#We specify the plot layout
return_dist_plot.update_layout(showlegend=False, title_text='Distribution of daily Repsol Returns', width=1000, height=500)
#Printing the plot
return_dist_plot.show()

In [ ]:
#This code uses the pacf() function from the tsa.stattools module of the statsmodels library (sm) to compute the autocorrelation function.
plot_pacf(returns_repsol**2,method="yw")
#Print the visualization
plt.show()

This graph shows the autocorrelation of squared returns, this positive result will let us to predict volatility using a GARCH model, or either adding the previous volatility as a feature to our XGBoost model.

<a id='gar'></a>
# 5. GARCH (4,4) vs XGBoost

## 5.1 GARCH

In [ ]:
#Define a GARCH model (4,4) that uses a ged distribution
model = arch_model(returns_repsol,dist="ged", vol = 'GARCH', p=4, q=4)
#Fit the model
model_fit = model.fit(disp='off')
#Summary of the model
model_fit.summary()

In [ ]:
#Define the full serie as the previosly defined model
full_serie_garch = arch_model(returns_repsol,dist="ged", vol = 'GARCH', p=4, q=4)
#Fitting the model for the full serie
model_fit_full_serie = full_serie_garch.fit(disp='off')
#We will plot against the rolling volatility
rolling_vol = abs(returns_repsol.rolling(window=22, min_periods=22).std().dropna())
#Concatenating the true values, and trained values in our model
garch_and_rolling_std = pd.concat([pd.DataFrame(model_fit_full_serie.conditional_volatility),rolling_vol.dropna()], axis=1).dropna()
#Plotting it
garch_and_rolling_std_plot = px.line(garch_and_rolling_std, title = 'GARCH vs rolling volatility of daily returns TRAIN', width=1000, height=500)
#Printing the plot
garch_and_rolling_std_plot.show()

In [ ]:
#Using a numeric range of 251 to fill a list of predicted values, for each day we are fitting a new model with the same parameters, but adding the last day.
test_size = 251
rolling_predictions = []

for i in range(test_size):
    train = returns_repsol[:-(test_size-i)]
    model = arch_model(train,dist="ged", vol = 'GARCH', p=4, q=4)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=1, reindex = False)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1,:][0]))

In [ ]:
#Transforming it to a serie 
rolling_predictions = pd.Series(rolling_predictions, index= returns_repsol.dropna().index[-test_size:])
#Setting plot parameters
plt.figure(figsize=(10,4))
#True data 
true, = plt.plot((rolling_vol)[-test_size:])
#Predicted data
preds, = plt.plot(rolling_predictions)
#Plot of the data
plt.title('Volatility Prediction for the next 251 Trading Days - Rolling Forecast TEST with GARCH(4,4)', fontsize=20)
#Add legend
plt.legend(['True Volatility', 'Predicted Volatility'], fontsize=16)

## 5.2 XGBoost

In [ ]:
#Read the raw data again, we will be transforming it in a different way
repsol_stock_raw_data_for_ml = pd.read_csv("/kaggle/input/repsol-stock-data-20-years/RepsolStockData20Years.csv")
#We only need the date and close columns
returns_repsol_for_ml = repsol_stock_raw_data_for_ml[["date","close"]]
#We change the close column for the percentual change calculation
returns_repsol_for_ml["close"] = 100 * repsol_stock_raw_data.close.pct_change().dropna()
#Drop 0 returns
returns_repsol_for_ml  = returns_repsol_for_ml.drop(returns_repsol_for_ml[returns_repsol_for_ml["close"] == 0].index)
#Drop N/A
returns_repsol_for_ml = returns_repsol_for_ml.dropna()
#As we will be comparing it to the rolling volatility of 22 days, we will transform our target to that
returns_repsol_for_ml["close"] = abs(returns_repsol_for_ml["close"].rolling(window=22, min_periods=22).std().dropna())
#Convert the date column to datetime format
returns_repsol_for_ml["date"] = pd.to_datetime(returns_repsol_for_ml["date"])
#Rename the dataframe
serie_for_xgboost = returns_repsol_for_ml
#Set the test size
test_size = 251
#Split train and test
train_ml = serie_for_xgboost[:-(test_size)].dropna()
test_ml = serie_for_xgboost[-(test_size):].dropna()

In [ ]:
#UDF for extracting features from date
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
#Creating the features for the train and test sets
X_train, y_train = create_features(train_ml, label="close")
X_test, y_test = create_features(test_ml, label="close")

In [ ]:
#Creating an aditional feature that uses the 4 previous days of rolling volatility, incorporating the autoregressive component to our ML model
X_train['prev1']=train_ml['close'].shift(1)
X_test['prev1']=test_ml['close'].shift(1)
X_train['prev2'] =train_ml['close'].shift(2)
X_test['prev2']=test_ml['close'].shift(2)
X_train['prev3'] =train_ml['close'].shift(3)
X_test['prev3']=test_ml['close'].shift(3)
X_train['prev4'] =train_ml['close'].shift(4)
X_test['prev4']=test_ml['close'].shift(4)

In [ ]:
#Defining and fitting the model
reg = xgb.XGBRegressor(n_estimators=1000,early_stopping_rounds=50,)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
       verbose=False)

In [ ]:
#Plot of feature importance
_ = plot_importance(reg, height=0.9)

In [ ]:
#Predicting with our model for both the train and test data
train_ml["Predictions"] = reg.predict(X_train)
test_ml['Prediction'] = reg.predict(X_test)

In [ ]:
#Creating the dataframe with both real and predicted vol
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(train_ml["Predictions"]),list(train_ml["close"]))], axis=1).dropna().reset_index()
#Renaming columns
XGBoost_and_rolling.rename(columns={"index":"Real_Volatility",0:"Predicted Volatility"}, inplace=True)
XGBoost_and_rolling.head(3)

In [ ]:
#Plotting the predictions of the training data
XGBoost_and_rolling = pd.concat([pd.DataFrame(list(train_ml["Predictions"]),list(train_ml["close"]))], axis=1).dropna().reset_index()
XGBoost_and_rolling.rename(columns={"index":"Real_Volatility",0:"Predicted Volatility"}, inplace=True)
XGBoost_and_rolling = px.line(XGBoost_and_rolling, title = 'XGBOOST vs rolling volatility of daily returns TRAIN', width=1000, height=500)
XGBoost_and_rolling.show()

In [ ]:
#Plotting the predictions for the test data
plt.figure(figsize=(10,4))
true, = plt.plot(test_ml["close"])
preds, = plt.plot(test_ml['Prediction'])
plt.title('Volatility Prediction for the next 251 Trading Days - Rolling Forecast TEST with XGBOOST', fontsize=20)
plt.legend(['True Returns', 'Predicted Volatility'], fontsize=16)

<a id='res'></a>
# 6. Results

Metrics: 
<br/><br/>


Root Mean Square Error, RMSE: Square root of the mean of the difference between
the actual data points and the squared prediction value. It penalizes
greater or extreme differences more.


The RMSE of the model is $RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$.

where n is the number of samples, $y_i$ is the true value for the ith sample, and $\hat{y}_i$ is the predicted value for the ith sample.

<br/><br/>
Mean Absolute Percentage Error, MAPE: It allows to measure
errors relative to the magnitude of the real value.


The MAPE of the model is $\text{MAPE} = \frac{100}{n} \sum_{i=1}^{n} \frac{|A_i - F_i|}{|A_i|}$

where $A_i$ is the actual value of the i-th sample, $F_i$ is the forecasted value of the i-th sample, and $n$ is the total number of samples.


In [ ]:
RMSE_Serie = mean_squared_error(garch_and_rolling_std["close"],garch_and_rolling_std["cond_vol"],squared=False)
MAPE_Serie = mean_absolute_percentage_error(garch_and_rolling_std["close"], garch_and_rolling_std["cond_vol"])
print(f"The RMSE of our GARCH model in the full serie data is {round(RMSE_Serie,4)}")
print(f"The MAPE of our GARCH model in the full serie data is {round(MAPE_Serie*100,2)}%")

In [ ]:
true_vol = rolling_vol[-test_size:]
pred_vol = rolling_predictions
RMSE = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our GARCH model in the predicted data is {round(RMSE,4)}")
print(f"The MAPE of our GARCH model in the predicted data is {round(MAPE*100,2)}%")

In [ ]:
RMSE_Serie_XG = mean_squared_error(train_ml["close"],train_ml["Predictions"],squared=False)
MAPE_Serie_XG = mean_absolute_percentage_error(train_ml["close"], train_ml["Predictions"])
print(f"The RMSE of our XGBOOST model in the full serie data is {round(RMSE_Serie_XG,4)}")
print(f"The MAPE of our XGBOOST model in the full serie data is {round(MAPE_Serie_XG*100,2)}%")

In [ ]:
true_vol = test_ml['Prediction']
pred_vol = test_ml["close"]
RMSE_XG = mean_squared_error(true_vol, pred_vol,squared=False)
MAPE_XG = mean_absolute_percentage_error(true_vol, pred_vol)
print(f"The RMSE of our XGBOOST model in the predicted data is {round(RMSE_XG,4)}")
print(f"The MAPE of our XGBOOST model in the predicted data is {round(MAPE_XG*100,2)}%")

In [ ]:
print(tabulate([['MAPE',round(MAPE_Serie*100,2),round(MAPE_Serie_XG*100,2),round(MAPE*100,2),round(MAPE_XG*100,2)]],headers = ['GARCH TRAIN %', 'XGBOOST TRAIN %', 'GARCH PREDICTIONS %','XGBOOST PREDICTIONS %'],tablefmt = 'fancy_grid',stralign='center',numalign='center',floatfmt=".2f"))

Our study has shown that the GARCH model can effectively predict the monthly rolling volatility of Repsol's stock. However, other modern approaches like XGBoost can improve upon this, as seen in our results where the MAPE was reduced by half to 4.43%. This project showcases the utility of both Machine Learning and GARCH for forecasting stock volatility, and underscores the importance of taking into account the time-varying nature of volatility in financial analysis. While this findings are promising, further research is needed to test these theories across a wider range of stocks, markets, and models in order to reach more general conclusions.

I really hope that this notebook was useful and interesting to anyone reading.